In [324]:
import json
from pprint import pprint
import pandas as pd
import os
import pickle
import numpy as np
import ipywidgets as widgets
from IPython.display import display

data = open("D:\Anaconda\progetto\FDA\players_advancedstats_Serie_A_2020_2021", "rb")
players = open("D:\Anaconda\progetto\FDA\players_Serie_A_2020_2021", "rb")
ad_st = pickle.load(data)
players = pickle.load(players)

In [325]:
CODE_FOLDER = os.getcwd()
DATA_FOLDER = os.getcwd().replace("code", "data")

In [326]:
df = pickle.load(open( r"C:\Users\mauro\Downloads\data\preprocessing_dataset", "rb"))
df.shape

(752, 13)

In [327]:
df.columns

Index(['team_id', 'team_name', 'main_scheme', 'player_id', 'player_name',
       'height', 'weight', 'age', 'birth_area', 'passport_area', 'foot',
       'main_role', 'contractExpiration'],
      dtype='object')

In [328]:
WIDGET_TYPE = {
    "category":["team_name", "main_scheme", "birth_area", "passport_area", "foot", "main_role"],
    "numeric":["age", "height", "weight", "contractExpiration"]
}
widgets_dict = {}
for w_type in WIDGET_TYPE.keys():
    for col in WIDGET_TYPE[w_type]:
        options_list = [x for x in list(df[col].unique()) if x == x and x is not None]
        if w_type == "category":
            widgets_dict[col] = widgets.SelectMultiple(options=sorted(options_list), description=col)
        elif w_type == "numeric":
            min_, max_ = df[df[col] > 0][col].min(), df[df[col] > 0][col].max()
            if col == "contractExpiration":
                min_ = 2021
            widgets_dict[col] = widgets.IntRangeSlider(value=[min_, max_], min=min_, max=max_, step=1, description=col)
        display(widgets_dict[col])

w = widgets.Dropdown(options=df["player_name"].sort_values(), disabled=False)
display(w)

SelectMultiple(description='team_name', options=('Atalanta', 'Benevento', 'Bologna', 'Cagliari', 'Crotone', 'F…

SelectMultiple(description='main_scheme', options=('3-4-1-2', '3-4-2-1', '3-4-3', '3-5-2', '4-2-3-1', '4-3-2-1…

SelectMultiple(description='birth_area', options=('AGO', 'ALB', 'ARG', 'ARM', 'AUT', 'BEL', 'BGR', 'BIH', 'BRA…

SelectMultiple(description='passport_area', options=('AGO', 'ALB', 'ARG', 'ARM', 'AUT', 'BEL', 'BEN', 'BFA', '…

SelectMultiple(description='foot', options=('both', 'left', 'right'), value=())

...

IntRangeSlider(value=(15, 43), description='age', max=43, min=15)

IntRangeSlider(value=(163, 202), description='height', max=202, min=163)

IntRangeSlider(value=(58, 96), description='weight', max=96, min=58)

IntRangeSlider(value=(2021, 2026), description='contractExpiration', max=2026, min=2021)

Dropdown(options=('A. Adorante', 'A. Arnofoli', 'A. Baldursson', 'A. Bane', 'A. Barreca', 'A. Barák', 'A. Basi…

In [329]:
player_selected = w.value

In [330]:
aux = df.copy()
for col in widgets_dict.keys():
    if col in WIDGET_TYPE["category"]:
        values = list(widgets_dict[col].value)
        if len(values) > 0:
            query = col + " in " + str(values)
            print(query)
            aux = aux.query(query)
    elif col in WIDGET_TYPE["numeric"]:
        query = col + " >= " + str(widgets_dict[col].value[0]) + " and " + col + " <= " + str(widgets_dict[col].value[1])
        print(query)
        aux = aux.query(query)

age >= 19 and age <= 29
height >= 178 and height <= 202
weight >= 58 and weight <= 96
contractExpiration >= 2021 and contractExpiration <= 2025


In [331]:
aux['main_role'] = aux['main_role'].fillna('dato non disponibile')
aux

,team_id,team_name,main_scheme,player_id,player_name,height,weight,age,birth_area,passport_area,foot,main_role,contractExpiration
0,3157.0,Milan,4-2-3-1,582906,P. Kalulu,179,69,20,FRA,COD,right,Right Centre Back,2025.0
1,3157.0,Milan,4-2-3-1,558720,D. Maldini,181,70,19,ITA,VEN,right,Left Attacking Midfielder,2024.0
7,3157.0,Milan,4-2-3-1,15524,H. Çalhanoğlu,178,76,27,TUR,TUR,right,Attacking Midfielder,2021.0
8,3157.0,Milan,4-2-3-1,25504,S. Meïté,187,91,27,FRA,CIV,right,Right Centre Midfielder,2023.0
9,3157.0,Milan,4-2-3-1,69616,A. Rebić,185,78,27,HRV,HRV,right,Left Attacking Midfielder,2025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,3315.0,Sassuolo,4-2-3-1,286226,M. Locatelli,185,75,23,ITA,ITA,right,Left Defensive Midfielder,2023.0
539,3315.0,Sassuolo,4-2-3-1,403449,Rogério,178,70,23,BRA,BRA,left,Left Back,2024.0
540,3315.0,Sassuolo,4-2-3-1,425915,M. Müldür,188,74,22,TUR,TUR,right,Right Back,2024.0
542,3315.0,Sassuolo,4-2-3-1,518046,S. Turati,189,85,19,ITA,ITA,right,dato non disponibile,2025.0


In [332]:
pd.set_option('display.max_rows', aux.shape[0]+1)
aux.sort_values('player_name')

,team_id,team_name,main_scheme,player_id,player_name,height,weight,age,birth_area,passport_area,foot,main_role,contractExpiration
238,3166.0,Bologna,4-2-3-1,476177,A. Baldursson,180,76,19,ISL,ISL,right,Left Defensive Midfielder,2025.0
313,3176.0,Fiorentina,3-5-2,300373,A. Barreca,183,70,26,ITA,ITA,left,Left Wingback,2023.0
422,3194.0,Hellas Verona,3-4-2-1,265172,A. Barák,190,86,26,CZE,CZE,left,Attacking Midfielder,2022.0
133,3161.0,Internazionale,3-5-2,405597,A. Bastoni,190,75,22,ITA,ITA,left,Left Centre Back (3 at the back),2023.0
335,3185.0,Torino,3-5-2,21234,A. Belotti,181,84,27,ITA,ITA,right,Striker,2022.0
345,3185.0,Torino,3-5-2,491451,A. Buongiorno,194,80,21,ITA,ITA,left,Left Centre Back (3 at the back),2024.0
283,3173.0,Cagliari,3-5-2,239289,A. Calabresi,186,75,25,ITA,ITA,right,Right Centre Back (3 at the back),2022.0
279,3173.0,Cagliari,3-5-2,134441,A. Cerri,194,91,25,ITA,ITA,right,Striker,2024.0
94,3160.0,Parma,4-3-3,134419,A. Conti,184,76,27,ITA,ITA,right,Right Back,2022.0
97,3160.0,Parma,4-3-3,135150,A. Cornelius,193,91,28,DNK,DNK,left,Striker,2022.0


In [333]:
print("There are " + str(len(aux)) + " players selected!")

There are 316 players selected!


In [334]:
players_selected = []
for n in aux['player_name']:
    players_selected.append(n)

In [335]:
row_player_selected = df.loc[df['player_name']==player_selected]
new = [row_player_selected, aux]
aux = pd.concat(new, axis=0)
aux.drop_duplicates("player_name", keep = "last", inplace = True)
aux


,team_id,team_name,main_scheme,player_id,player_name,height,weight,age,birth_area,passport_area,foot,main_role,contractExpiration
0,3157.0,Milan,4-2-3-1,582906,P. Kalulu,179,69,20,FRA,COD,right,Right Centre Back,2025.0
1,3157.0,Milan,4-2-3-1,558720,D. Maldini,181,70,19,ITA,VEN,right,Left Attacking Midfielder,2024.0
7,3157.0,Milan,4-2-3-1,15524,H. Çalhanoğlu,178,76,27,TUR,TUR,right,Attacking Midfielder,2021.0
8,3157.0,Milan,4-2-3-1,25504,S. Meïté,187,91,27,FRA,CIV,right,Right Centre Midfielder,2023.0
9,3157.0,Milan,4-2-3-1,69616,A. Rebić,185,78,27,HRV,HRV,right,Left Attacking Midfielder,2025.0
10,3157.0,Milan,4-2-3-1,134174,Samu Castillejo,182,67,26,ESP,ESP,left,Right Attacking Midfielder,2023.0
11,3157.0,Milan,4-2-3-1,146588,A. Romagnoli,185,75,26,ITA,ITA,left,Left Centre Back,2022.0
12,3157.0,Milan,4-2-3-1,254493,R. Krunić,184,74,27,BIH,BIH,right,Attacking Midfielder,2024.0
14,3157.0,Milan,4-2-3-1,285109,F. Kessié,183,88,24,CIV,CIV,right,Left Defensive Midfielder,2022.0
15,3157.0,Milan,4-2-3-1,291591,F. Tomori,185,78,23,CAN,XEN,right,Left Centre Back,2024.0


In [336]:
df = aux
advanced_stats = pickle.load(open("D:\Anaconda\progetto\FDA\players_advancedstats_Serie_A_2020_2021", "rb"))
advanced_stats = pd.DataFrame(advanced_stats).transpose().dropna(how="all")

average_stats = pd.json_normalize(advanced_stats["average"])
average_stats.columns = average_stats.columns + "_P90"
average_stats.index = advanced_stats.index

percent_stats = pd.json_normalize(advanced_stats["percent"])
percent_stats.columns = "%_" + percent_stats.columns
percent_stats.index = advanced_stats.index

stats = pd.merge(average_stats, percent_stats, left_index=True, right_index=True)
stats = pd.merge(advanced_stats[["competitionId", "seasonId"]], stats, left_index=True, right_index=True)

final_df = pd.merge(df, stats, left_on="player_id", right_index=True, how="left")

In [337]:
final_df

,team_id,team_name,main_scheme,player_id,player_name,height,weight,age,birth_area,passport_area,...,%_dribblesAgainstWon,%_fieldAerialDuelsWon,%_gkSaves,%_gkSuccessfulExits,%_gkAerialDuelsWon,%_newDuelsWon,%_newDefensiveDuelsWon,%_newOffensiveDuelsWon,%_newSuccessfulDribbles,%_successfulLateralPasses
0,3157.0,Milan,4-2-3-1,582906,P. Kalulu,179,69,20,FRA,COD,...,52.94,37.93,0.00,0.0,0.00,60.00,70.24,56.25,66.67,90.86
1,3157.0,Milan,4-2-3-1,558720,D. Maldini,181,70,19,ITA,VEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3157.0,Milan,4-2-3-1,15524,H. Çalhanoğlu,178,76,27,TUR,TUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3157.0,Milan,4-2-3-1,25504,S. Meïté,187,91,27,FRA,CIV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3157.0,Milan,4-2-3-1,69616,A. Rebić,185,78,27,HRV,HRV,...,27.27,40.68,0.00,0.0,0.00,40.98,46.94,38.42,53.47,66.67
10,3157.0,Milan,4-2-3-1,134174,Samu Castillejo,182,67,26,ESP,ESP,...,42.50,28.00,0.00,0.0,0.00,43.13,50.46,42.86,47.06,77.70
11,3157.0,Milan,4-2-3-1,146588,A. Romagnoli,185,75,26,ITA,ITA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,3157.0,Milan,4-2-3-1,254493,R. Krunić,184,74,27,BIH,BIH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,3157.0,Milan,4-2-3-1,285109,F. Kessié,183,88,24,CIV,CIV,...,48.68,54.63,0.00,0.0,0.00,48.95,60.55,38.33,50.00,94.97
15,3157.0,Milan,4-2-3-1,291591,F. Tomori,185,78,23,CAN,XEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
import ipywidgets as widgets

MACRO_AREAS = {
    "ATTACK":["%_successfulLinkupPlays", "attackingActions_P90", "foulsSuffered_P90", 
              "linkupPlays_P90", "offsides_P90", "receivedPass_P90", "successfulAttackingActions_P90",
              "successfulLinkupPlays_P90", "touchInBox_P90"],
    "CARDS":["%_yellowCardsPerFoul", "directRedCards_P90", "redCards_P90", "yellowCards_P90"],
    "CROSS":["%_successfulCrosses", "crosses_P90", "successfulCrosses_P90"],
    "DEFENSE":["%_dribblesAgainstWon", "%_successfulSlidingTackles", "ballLosses_P90", 
               "ballRecoveries_P90", "clearances_P90", "counterpressingRecoveries_P90",
               "dangerousOpponentHalfRecoveries_P90", "dangerousOwnHalfLosses_P90",
               "defensiveActions_P90", "dribblesAgainst_P90", "dribblesAgainstWon_P90",
               "interceptions_P90", "losses_P90", "missedBalls_P90", "opponentHalfRecoveries_P90",
               "ownHalfLosses_P90", "shotsBlocked_P90", "slidingTackles_P90", 
               "successfulDefensiveAction_P90", "successfulSlidingTackles_P90"],
    "DRIBBLING":["%_newSuccessfulDribbles", "%_successfulDribbles", "dribbles_P90",
                 "newSuccessfulDribbles_P90", "successfulDribbles_P90"],
    "DUELS":["%_aerialDuelsWon", "%_defensiveDuelsWon", "%_duelsWon",
             "%_fieldAerialDuelsWon", "%_newDefensiveDuelsWon", "%_newDuelsWon",
             "%_newOffensiveDuelsWon", "%_offensiveDuelsWon", "aerialDuels_P90",
             "defensiveDuels_P90", "defensiveDuelsWon_P90", "duels_P90",
             "duelsWon_P90", "fieldAerialDuels_P90", "fieldAerialDuelsWon_P90",
             "fouls_P90", "looseBallDuels_P90", "looseBallDuelsWon_P90",
             "newDefensiveDuelsWon_P90", "newDuelsWon_P90", "newOffensiveDuelsWon_P90",
             "offensiveDuels_P90", "offensiveDuelsWon_P90"],
    "GOALKEEPER":["%_gkAerialDuelsWon", "%_gkSaves", "%_gkSuccessfulExits", 
                  "%_successfulGoalKicks", "gkAerialDuels_P90", "gkAerialDuelsWon_P90",
                  "gkConcededGoals_P90", "gkExits_P90", "gkSaves_P90", "gkShotsAgainst_P90",
                  "gkSuccessfulExits_P90", "goalKicks_P90", "goalKicksLong_P90",
                  "goalKicksShort_P90", "successfulGoalKicks_P90", "xgSave_P90"],
    "PASSES":["%_successfulBackPasses", "%_successfulForwardPasses", "%_successfulKeyPasses",
              "%_successfulLongPasses", "%_successfulPasses", "%_successfulPassesToFinalThird",
              "%_successfulProgressivePasses", "%_successfulShotAssists", 
              "%_successfulSmartPasses", "%_successfulThroughPasses", 
              "%_successfulVerticalPasses", "assists_P90", "backPasses_P90",
              "dribbleDistanceFromOpponentGoal_P90", "forwardPasses_P90", "keyPasses_P90",
              "lateralPasses_P90", "longPasses_P90", "longPassLength_P90", "passes_P90",
              "passesToFinalThird_P90", "passLength_P90", "progressivePasses_P90",
              "secondAssists_P90", "shotAssists_P90", "shotOnTargetAssists_P90", 
              "smartPasses_P90", "successfulBackPasses_P90", "successfulForwardPasses_P90",
              "successfulKeyPasses_P90", "successfulLateralPasses_P90", "successfulLongPasses_P90",    
              "successfulPasses_P90", "successfulPassesToFinalThird_P90", "successfulProgressivePasses_P90",
              "successfulSmartPasses_P90", "successfulThroughPasses_P90", "successfulVerticalPasses_P90",
              "thirdAssists_P90", "throughPasses_P90", "verticalPasses_P90",
              "xgAssist_P90", "%_successfulLateralPasses"],
    "PHYSICS":["accelerations_P90", "progressiveRun_P90"],
    "SET_PIECES":["%_directFreeKicksOnTarget", "%_penaltiesConversion", "corners_P90", 
                  "directFreeKicks_P90", "directFreeKicksOnTarget_P90", "freeKicks_P90",
                  "freeKicksOnTarget_P90", "penalties_P90", "successfulPenalties_P90"],
    "SHOTS":["%_goalConversion", "%_headShotsOnTarget", "%_shotsOnTarget", "goals_P90", 
             "headShots_P90", "shots_P90", "shotsOnTarget_P90", "xgShot_P90"]
}


        
checkboxes = [widgets.Checkbox(value=False, description=label) for label in MACRO_AREAS]
output = widgets.VBox(children=checkboxes)
display(output)


    




In [339]:
selected_data = []

for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        selected_data = selected_data + [checkboxes[i].description]

data_clustering = []
for key in selected_data:
    data_clustering.append(MACRO_AREAS[key])

        
data_list = []

for n in data_clustering:
    for item in n:
        data_list.append(item)




checkboxes_1 = [widgets.Checkbox(value=False, description=label) for label in data_list]
output = widgets.VBox(children=checkboxes_1)
display(output)

         

        
    

        
                
    
    



In [340]:
selected_data

['CROSS', 'DEFENSE']

In [341]:
data_for_clustering = []
for i in range(0, len(checkboxes_1)):
    if checkboxes_1[i].value == True:
        data_for_clustering = data_for_clustering + [checkboxes_1[i].description]

In [342]:
data_for_clustering

['crosses_P90', 'successfulCrosses_P90', 'dribblesAgainstWon_P90']

In [343]:
df_for_clustering = final_df[data_for_clustering]
players_analysis = aux['player_name']
df_for_clustering

,crosses_P90,successfulCrosses_P90,dribblesAgainstWon_P90
0,1.84,0.34,2.07
1,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,3.15,1.18,0.48
10,2.59,0.55,1.33
11,NaN,NaN,NaN
12,NaN,NaN,NaN
14,0.44,0.06,1.09
15,NaN,NaN,NaN


In [344]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
df_for_clustering = df_for_clustering.fillna(0)
pca = PCA(n_components = 2)

X = pd.DataFrame(pca.fit_transform(df_for_clustering))
X

,0,1
0,1.803669,1.018851
1,-0.695746,-0.200782
2,-0.695746,-0.200782
3,-0.695746,-0.200782
4,2.597105,-1.020531
5,2.246415,0.040387
6,-0.695746,-0.200782
7,-0.695746,-0.200782
8,0.119894,0.644598
9,-0.695746,-0.200782


In [345]:
b = widgets.Dropdown(options=['1','2','3','4','5','6','7','8','9','10'], value='3',description='cluster da creare:',  disabled=False)
display(b)


Dropdown(description='cluster da creare:', index=2, options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'…

In [346]:
n_cluster = b.value
n_cluster = int(n_cluster)

In [347]:
kmeans = KMeans(n_cluster)
kmeans = kmeans.fit(X)
labels = kmeans.predict(X)
centroid = kmeans.cluster_centers_
clusters = kmeans.labels_.tolist()

In [348]:
analisi_finale = X
clusters = pd.DataFrame(clusters)
players = final_df['player_name']
data = [analisi_finale, clusters, players]
data_finale = pd.concat(data, axis=1)
d ={0:'x', 1:'y', 2: 'clusters', 'player_name':'name'}
data_finale = data_finale.rename(columns = d)
data_finale.columns.values[2] ='clusters'
data_finale = data_finale.dropna()
idx = data_finale.index[data_finale['name']==player_selected].tolist()
s = data_finale['clusters']
cluster = s[idx]
cluster


264    0.0
Name: clusters, dtype: float64

In [350]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go


fig = px.scatter(data_finale['x'], data_finale['y'], color = data_finale['clusters'], text = data_finale['name'])
pyo.plot(fig, filename = 'calcio.html')

'calcio.html'